# 2 — Implied Volatility

In this notebook we:

1. Define the notion of **implied volatility**.
2. Discuss why implied volatility is not given by a closed-form formula.
3. Implement a numerical solver for implied volatility using the **bisection method**.
4. Apply the solver to sample option quotes from the SQL database.
5. Explore how the Black–Scholes price changes with $\sigma$ and verify monotonicity.


In [1]:
import sys
from pathlib import Path

# Adjust this path if your project root is different
project_root = Path(r"C:\Users\Ruben\Desktop\Projects\OptionPricing")
if str(project_root) not in sys.path:
    sys.path.append(str(project_root))

from option_pricing.black_scholes import black_scholes_call, black_scholes_put
from option_pricing.db import get_connection
from option_pricing.utils import annualize_days

import math
import datetime as dt

print("Project root:", project_root)


Project root: C:\Users\Ruben\Desktop\Projects\OptionPricing


## 1. Definition of implied volatility

In the Black–Scholes model, the price of a European call option is

$$
C_{\text{BS}}(S_0, K, T, r, \sigma),
$$

and the price of a European put is

$$
P_{\text{BS}}(S_0, K, T, r, \sigma).
$$

Here the volatility $\sigma$ is an **input parameter**.

In real markets we observe the following quantities:

- $S_0$ — current underlying price,
- $K$ — strike price,
- $T$ — time to maturity,
- $r$ — risk-free rate,
- $C_{\text{mkt}}$ or $P_{\text{mkt}}$ — market price of the option (for example the mid of bid and ask).

The **implied volatility** $\sigma_{\text{imp}}$ is defined as the value of $\sigma$ such that

- for a call:
  $$
  C_{\text{BS}}(S_0, K, T, r, \sigma_{\text{imp}}) = C_{\text{mkt}},
  $$
- for a put:
  $$
  P_{\text{BS}}(S_0, K, T, r, \sigma_{\text{imp}}) = P_{\text{mkt}}.
  $$

There is no closed-form expression for $\sigma_{\text{imp}}$ in terms of the other variables, so we need to solve this equation **numerically**.


## 2. Monotonicity of option price in volatility

For both European calls and puts in the Black–Scholes model, keeping all other parameters fixed:

- the option price is a **strictly increasing function** of volatility $\sigma$.

Intuition:

- Higher volatility means a wider distribution of possible future prices.
- For options, this is beneficial, because payoffs are asymmetric (limited downside, unlimited upside for calls).
- Therefore, higher $\sigma$ increases the expected payoff under the risk-neutral measure.

Because of this monotonicity, for a **fixed** $(S_0, K, T, r)$, the function

$$
f(\sigma) = \text{BS\_price}(\sigma) - \text{market\_price}
$$

changes sign at most once, and we can use a simple **bisection method** to find the root $f(\sigma) = 0$.


In [2]:
def implied_vol_bisection(
    market_price: float,
    S0: float,
    K: float,
    T: float,
    r: float,
    option_type: str,
    sigma_lower: float = 1e-6,
    sigma_upper: float = 5.0,
    tol: float = 1e-6,
    max_iter: int = 100
) -> float:
    """
    Compute implied volatility using the bisection method.

    Parameters
    ----------
    market_price : float
        Observed market price of the option (for example mid of bid and ask).
    S0 : float
        Current underlying price.
    K : float
        Strike price.
    T : float
        Time to maturity in years.
    r : float
        Risk-free interest rate (continuously compounded).
    option_type : str
        'CALL' or 'PUT'.
    sigma_lower : float
        Lower bound for volatility search interval.
    sigma_upper : float
        Upper bound for volatility search interval.
    tol : float
        Tolerance for absolute pricing error.
    max_iter : int
        Maximum number of bisection iterations.

    Returns
    -------
    float
        Implied volatility sigma such that Black–Scholes price is close to market_price.

    Raises
    ------
    ValueError
        If the market price is outside the range of possible option values
        over the interval [sigma_lower, sigma_upper].
    """
    option_type = option_type.upper()
    if option_type not in ("CALL", "PUT"):
        raise ValueError("option_type must be 'CALL' or 'PUT'")

    def bs_price(sigma: float) -> float:
        if option_type == "CALL":
            return black_scholes_call(S0, K, T, r, sigma)
        else:
            return black_scholes_put(S0, K, T, r, sigma)

    # Check that market_price lies between prices at sigma_lower and sigma_upper
    price_lower = bs_price(sigma_lower)
    price_upper = bs_price(sigma_upper)

    if not (price_lower <= market_price <= price_upper):
        raise ValueError(
            f"Market price {market_price:.4f} is outside the range "
            f"[{price_lower:.4f}, {price_upper:.4f}] for sigma in "
            f"[{sigma_lower}, {sigma_upper}]."
        )

    for _ in range(max_iter):
        sigma_mid = 0.5 * (sigma_lower + sigma_upper)
        price_mid = bs_price(sigma_mid)

        # If we are close enough in price, stop
        if abs(price_mid - market_price) < tol:
            return sigma_mid

        # Decide which half of the interval to keep
        if price_mid < market_price:
            sigma_lower = sigma_mid
        else:
            sigma_upper = sigma_mid

    # If we reach here, we did not converge within max_iter
    return 0.5 * (sigma_lower + sigma_upper)


In [3]:
true_sigma = 0.30

S0_test = 100.0
K_test = 110.0
T_test = 0.75
r_test = 0.01

# Generate "market" price from the model
market_call_price = black_scholes_call(S0_test, K_test, T_test, r_test, true_sigma)

iv_estimate = implied_vol_bisection(
    market_price=market_call_price,
    S0=S0_test,
    K=K_test,
    T=T_test,
    r=r_test,
    option_type="CALL"
)

print(f"True sigma:       {true_sigma:.6f}")
print(f"Estimated sigma:  {iv_estimate:.6f}")
print(f"Absolute error:   {abs(iv_estimate - true_sigma):.6e}")


True sigma:       0.300000
Estimated sigma:  0.300000
Absolute error:   1.665436e-08


In [4]:
conn = get_connection()

rows = conn.execute("""
SELECT *
FROM option_quotes
WHERE underlying_symbol = 'AAPL'
ORDER BY option_type, strike, expiration_date
;
""").fetchall()

conn.close()

len(rows), [dict(rows[0]).keys()]


(2,
 [dict_keys(['id', 'underlying_symbol', 'quote_date', 'expiration_date', 'strike', 'option_type', 'bid', 'ask', 'mid', 'underlying_price', 'risk_free_rate', 'dividend_yield', 'source'])])

In [5]:
results = []

for row in rows:
    data = dict(row)

    S0_q = data["underlying_price"]
    K_q = data["strike"]
    r_q = data["risk_free_rate"]
    mid_q = data["mid"]
    option_type_q = data["option_type"]

    quote_date = dt.date.fromisoformat(data["quote_date"])
    expiration_date = dt.date.fromisoformat(data["expiration_date"])
    days_to_maturity = (expiration_date - quote_date).days
    T_q = annualize_days(days_to_maturity)

    try:
        iv = implied_vol_bisection(
            market_price=mid_q,
            S0=S0_q,
            K=K_q,
            T=T_q,
            r=r_q,
            option_type=option_type_q,
            sigma_lower=1e-4,
            sigma_upper=3.0,
            tol=1e-6,
            max_iter=100
        )
        status = "OK"
    except ValueError as e:
        iv = float("nan")
        status = f"Error: {e}"

    results.append(
        {
            "underlying": data["underlying_symbol"],
            "option_type": option_type_q,
            "strike": K_q,
            "quote_date": quote_date,
            "expiration_date": expiration_date,
            "mid_price": mid_q,
            "S0": S0_q,
            "T_years": T_q,
            "r": r_q,
            "implied_vol": iv,
            "status": status,
        }
    )

results


[{'underlying': 'AAPL',
  'option_type': 'CALL',
  'strike': 210.0,
  'quote_date': datetime.date(2025, 1, 3),
  'expiration_date': datetime.date(2025, 3, 21),
  'mid_price': 5.25,
  'S0': 212.2,
  'T_years': 0.21095890410958903,
  'r': 0.045,
  'implied_vol': 0.06880392422080038,
  'status': 'OK'},
 {'underlying': 'AAPL',
  'option_type': 'PUT',
  'strike': 210.0,
  'quote_date': datetime.date(2025, 1, 3),
  'expiration_date': datetime.date(2025, 3, 21),
  'mid_price': 4.93,
  'S0': 212.2,
  'T_years': 0.21095890410958903,
  'r': 0.045,
  'implied_vol': 0.17716933452785016,
  'status': 'OK'}]

In [6]:
import pandas as pd

df_iv = pd.DataFrame(results)
df_iv


,underlying,option_type,strike,quote_date,expiration_date,mid_price,S0,T_years,r,implied_vol,status
0,AAPL,CALL,210.0,2025-01-03,2025-03-21,5.25,212.2,0.210959,0.045,0.068804,OK
1,AAPL,PUT,210.0,2025-01-03,2025-03-21,4.93,212.2,0.210959,0.045,0.177169,OK


In [7]:
row0 = results[0]

S0_q = row0["S0"]
K_q = row0["strike"]
T_q = row0["T_years"]
r_q = row0["r"]
sigma_iv = row0["implied_vol"]
mid_q = row0["mid_price"]
option_type_q = row0["option_type"]

if option_type_q == "CALL":
    model_price = black_scholes_call(S0_q, K_q, T_q, r_q, sigma_iv)
else:
    model_price = black_scholes_put(S0_q, K_q, T_q, r_q, sigma_iv)

print(f"Market mid price:  {mid_q:.6f}")
print(f"Model price (IV):  {model_price:.6f}")
print(f"Difference:        {model_price - mid_q:.6e}")


Market mid price:  5.250000
Model price (IV):  5.249999
Difference:        -8.180804e-07


## 3. Numerical issues and limitations

Some practical points about implied volatility:

1. **No solution**  
   If the market price is below the intrinsic value or above the maximum BS price
   over the chosen volatility interval, the equation
   $$
   \text{BS\_price}(\sigma) = \text{market\_price}
   $$
   may have no solution.  
   In this case our function raises an error.

2. **Choice of bounds**  
   The interval $[10^{-6}, 5.0]$ for $\sigma$ is arbitrary but reasonable for many
   liquid equity options (0%–500% volatility).  
   In other markets or stressed scenarios, you may need wider bounds.

3. **Bid–ask spread**  
   Real options have a bid and an ask price.  
   Using the **mid** price for implied volatility is common but still arbitrary.
   Professional desks often compute implied volatilities for bid, mid and ask,
   or for the trade price.

4. **Model error**  
   Even if we find a $\sigma_{\text{imp}}$ that matches the market price,
   this does **not** mean the Black–Scholes model is "true".  
   Implied volatility is simply the volatility parameter that makes the model
   agree with this one price.

In notebook 3 we will use implied volatilities to study **Greeks** and see how
sensitivities depend on $\sigma$.


# Conclusion

In this notebook we:

- Defined implied volatility as the value of $\sigma$ that makes the Black–Scholes
  price equal to the market price.
- Used the monotonic relationship between option price and volatility to justify
  a simple bisection root-finding method.
- Implemented a robust implied volatility solver.
- Tested the solver on synthetic data with a known true volatility.
- Applied the solver to sample option quotes stored in the SQL database and
  verified that plugging the implied volatility back into the model reproduces
  the market price.

Next, in **notebook 3 — Greeks and Sensitivities**, we will:

- compute the main Greeks (delta, gamma, vega, theta, rho),
- study how they depend on $S_0$, $K$, $T$ and $\sigma$,
- and interpret what they mean in terms of risk and hedging.
